# Absolute Gain Calibration using Stopping Muons

This notebook facilitates the latter part of a workflow that takes cosmic muons entering the MicroBooNE detector
and selects those that stop.

In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
import matplotlib.dates as dts
import numpy as np
import pandas as pd
import itertools
import os
import ROOT
import datetime
from root_numpy import root2array, root2rec, tree2rec, array2root, tree2array
from scipy.optimize import curve_fit
from scipy.misc import factorial

In [ ]:
FIGPATH = "./figs/"
PLOTSTATUS = "[Run4a EXT Vali.]"

In [ ]:
plt.rcParams.update({"font.size": 18})

In [ ]:
#f = ROOT.TFile('mcc83_cosmic.root')
# f = ROOT.TFile('/home/david/data/MCC9/November/StopMu/stopmu_mcc9_data_v2.root')
# f = ROOT.TFile('./data/stopmu_ana.root')
# f = ROOT.TFile('./data/stopmu_ana_merged_3000files.root')
f = ROOT.TFile('./data/stopmu_ana_bnb_ext_run4a_validation.root')

f.cd('stopmu')
print f
# df = pd.DataFrame(tree2rec(f.Get('stopmu/reco_tree')))
df = pd.DataFrame(tree2array(f.Get('stopmu/reco_tree')).view(np.recarray))
print df.shape
df = df.query('_trk_start_z > 500. and _trk_end_z > 500.')
print df.shape

In [ ]:
for key in df.keys():
    print key

In [ ]:
# validate ACPT tagging is working correctly
dfAnode = df.query(
    "(_trk_start_y > 80 and _trk_end_y > -80 and _trk_end_x < _trk_start_x) or (_trk_start_y < 80 and _trk_end_y < -80 and _trk_end_x > _trk_start_x)"
)
dfCathode = df.query(
    "(_trk_start_y > 80 and _trk_end_y > -80 and _trk_end_x > _trk_start_x) or (_trk_start_y < 80 and _trk_end_y < -80 and _trk_end_x < _trk_start_x)"
)

In [ ]:
TMAX = 100
fig = plt.figure(figsize=(6, 6))
plt.hist(
    dfAnode["_delta_t_closest_flash"].values,
    bins=np.linspace(-TMAX, TMAX, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="Anode Crossing",
)
plt.hist(
    dfCathode["_delta_t_closest_flash"].values,
    bins=np.linspace(-TMAX, TMAX, 100),
    histtype="step",
    lw=2,
    color="r",
    normed=True,
    label="Cathode Crossing",
)
plt.xlabel("$\Delta$t with closest flash [$\mu$s]", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=1)
plt.gca().axes.get_yaxis().set_ticks([])
plt.show()

In [ ]:
def GetMIPFit(x):

    dqdx_v = x["_dqdx_tm_y"]
    rr_v = x["_rr_y"]

    rr_max = x["_trk_len"]

    if rr_max < 150.0:
        return -1, -1

    rr_MIP_idx_v = np.where((rr_v > 100) & (rr_v < 150))[0]
    if len(rr_MIP_idx_v) == 0:
        return -1, -1
    dqdx_MIP_v = [dqdx_v[i] for i in rr_MIP_idx_v]
    avg = np.average(dqdx_MIP_v)
    rms = np.std(dqdx_MIP_v)
    return avg, rms


def BraggArea(x, rmin, rmax):

    dqdx_v = x["_dqdx_tm_y"]
    rr_v = x["_rr_y"]
    mip = x["mip"]

    if mip < 0:
        return -1

    rr_BRAGG_idx_v = np.where((rr_v < rmax) & (rr_v > rmin))[0]
    area = 0
    for i in rr_BRAGG_idx_v:
        if i < len(rr_v) - 1:
            ampl = dqdx_v[i] - mip
            area += ampl * np.abs(rr_v[i] - rr_v[i + 1])
            # print '+= ampl = %.02f \t\t dr = %.02f'%(ampl,np.abs(rr_v[i]-rr_v[i+1]))
    return area / mip


def dQfit(x, a, b, c):
    return a * x ** 2 + b * x + c


def MIPfit(x):

    dqdx_v = x["_dqdx_tm_y"]
    rr_v = x["_rr_y"]
    mip = x["mip"]

    dqdx_v = dqdx_v.astype(float)
    rr_v = rr_v.astype(float)

    if len(rr_v) < 50:
        return -1

    rr_v_fit = []
    dqdx_trunc_v_fit = []
    for i, r in enumerate(rr_v):
        if r < 50:
            rr_v_fit.append(r)
            dqdx_trunc_v_fit.append(dqdx_v[i] / mip)

    try:
        popt, popv = curve_fit(dQfit, rr_v_fit, dqdx_trunc_v_fit)
        return popt[2]
    except:
        return -1

In [ ]:
df["mip"] = df.apply(lambda x: GetMIPFit(x)[0], axis=1)
df["miperr"] = df.apply(lambda x: GetMIPFit(x)[1], axis=1)

In [ ]:
df["mipfit"] = df.apply(lambda x: MIPfit(x), axis=1)

In [ ]:
df["BA_000_025"] = df.apply(lambda x: BraggArea(x, 0, 25), axis=1)
df["BA_025_050"] = df.apply(lambda x: BraggArea(x, 25, 50), axis=1)
df["BA_000_050"] = df.apply(lambda x: BraggArea(x, 0, 50), axis=1)
df["BA_050_100"] = df.apply(lambda x: BraggArea(x, 50, 100), axis=1)

In [ ]:
def dt(x):
    return np.abs(x["_delta_t_closest_flash"])

In [ ]:
df["dt"] = df.apply(lambda x: dt(x), axis=1)

In [ ]:
def SElen(xs, xe, ys, ye, zs, ze):
    dd = (df[xs] - df[xe]) ** 2 + (df[ys] - df[ye]) ** 2 + (df[zs] - df[ze]) ** 2
    return np.sqrt(dd)

In [ ]:
df["SElen"] = SElen(
    "_trk_start_x",
    "_trk_end_x",
    "_trk_start_y",
    "_trk_end_y",
    "_trk_start_z",
    "_trk_end_z",
)

In [ ]:
print df.shape
df = df.query('BA_000_050 != -1 and BA_050_100 != -1')
print df.shape

In [ ]:
print df.shape
df = df.query('BA_000_025 != -1 and BA_025_050 != -1')
print df.shape

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["mip"].values,
    bins=np.linspace(100, 450, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Average dQ/dx in RR [100,150] cm", fontsize=20, fontweight="bold")
# plt.axvline(200,color='k',linestyle='--',lw=3)
# plt.axvline(350,color='k',linestyle='--',lw=3)
plt.grid()
plt.legend(loc=1)
plt.show()

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["mipfit"].values,
    bins=np.linspace(0, 3, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Quadratic Fit Intercept @ RR = 0", fontsize=20, fontweight="bold")
plt.title("Y0", fontsize=20, fontweight="bold")
plt.axvline(1.3, color="k", linestyle="--", lw=3)
plt.axvline(2.5, color="k", linestyle="--", lw=3)
plt.grid()
plt.legend(loc=1)
plt.savefig("./figs/y0.png", bbox_inches="tight")
plt.savefig("./figs/y0.pdf", bbox_inches="tight")
plt.show()

mipeA = df["miperr"].values / df["mip"].values

fig = plt.figure(figsize=(6, 6))
plt.hist(
    mipeA,
    bins=np.linspace(0, 1, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("MIP $\sigma/\mu$ in RR [100,150]", fontsize=20, fontweight="bold")

plt.axvline(0.1, color="k", linestyle="--", lw=3)
plt.xlim([0, 0.3])
plt.grid()
plt.legend(loc=1)
plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["BA_000_050"].values,
    bins=np.linspace(-30, 30, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Bragg Area in [0,50] cm range", fontsize=20, fontweight="bold")
plt.title("BA0", fontsize=20, fontweight="bold")
# plt.axvline(10,color='k',linestyle='--',lw=3)
# plt.axvline(20,color='k',linestyle='--',lw=3)
plt.grid()
plt.legend(loc=2)
plt.savefig("./figs/BA_000_050.png", bbox_inches="tight")
plt.savefig("./figs/BA_000_050.pdf", bbox_inches="tight")
plt.show()

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["BA_050_100"].values,
    bins=np.linspace(-30, 30, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Bragg Area in [50,100] cm range", fontsize=20, fontweight="bold")
plt.title("BA1", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=2)
# plt.ylim([0,300])
plt.savefig("./figs/BA_050_100.png", bbox_inches="tight")
plt.savefig("./figs/BA_050_100.pdf", bbox_inches="tight")

plt.show()

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["BA_000_025"].values,
    bins=np.linspace(-30, 30, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Bragg Area in [0,25] cm range", fontsize=20, fontweight="bold")
plt.title("BA 00-25", fontsize=20, fontweight="bold")
plt.grid()
plt.axvline(3, color="k", linestyle="--", lw=3)
plt.axvline(20, color="k", linestyle="--", lw=3)
plt.legend(loc=2)
# plt.ylim([0,300])
plt.savefig("./figs/BA_000_025.png", bbox_inches="tight")
plt.savefig("./figs/BA_000_025.pdf", bbox_inches="tight")

plt.show()

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["BA_025_050"].values,
    bins=np.linspace(-30, 30, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Bragg Area in [25,50] cm range", fontsize=20, fontweight="bold")
plt.title("BA 25-50", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=2)
# plt.ylim([0,300])
plt.savefig("./figs/BA_025_050.png", bbox_inches="tight")
plt.savefig("./figs/BA_025_050.pdf", bbox_inches="tight")

plt.show()

In [ ]:
TMAX = 50

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["_delta_t_closest_flash"].values,
    bins=np.linspace(-TMAX, TMAX, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("$\Delta$t with closest flash [$\mu$s]", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=1)
plt.axvline(14, color="k", linestyle="--", lw=3)
plt.gca().axes.get_yaxis().set_ticks([])
plt.savefig("./figs/delta_t_closest_flash.png", bbox_inches="tight")
plt.savefig("./figs/delta_t_closest_flash.pdf", bbox_inches="tight")

plt.show()

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["_trk_end_y"].values,
    bins=np.linspace(-120, 0, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("End Y Coordinate [cm]", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=1)
plt.axvline(-98, color="k", linestyle="--", lw=3)
plt.gca().axes.get_yaxis().set_ticks([])
plt.savefig("./figs/end_y_coord.png", bbox_inches="tight")
plt.savefig("./figs/end_y_coord.pdf", bbox_inches="tight")

plt.show()

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["_trk_end_x"].values,
    bins=np.linspace(240, 260, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("End X Coordinate [cm]", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=9)
# plt.axvline(-40,color='k',linestyle='--',lw=3)
# plt.axvline(300,color='k',linestyle='--',lw=3)
plt.gca().axes.get_yaxis().set_ticks([])
plt.savefig("./figs/end_x_coord.png", bbox_inches="tight")
plt.savefig("./figs/end_x_coord.pdf", bbox_inches="tight")

plt.show()

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["_trk_end_z"].values,
    bins=np.linspace(500, 1050, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("End Z Coordinate [cm]", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=3)
# plt.axvline(697,color='k',linestyle='--',lw=3)
# plt.axvline(740,color='k',linestyle='--',lw=3)
plt.gca().axes.get_yaxis().set_ticks([])
plt.savefig("./figs/end_z_coord.png", bbox_inches="tight")
plt.savefig("./figs/end_z_coord.pdf", bbox_inches="tight")

plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["_trk_start_y"].values,
    bins=np.linspace(80, 120, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Start Y Coordinate [cm]", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=2)
plt.gca().axes.get_yaxis().set_ticks([])
plt.show()

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["_trk_start_x"].values,
    bins=np.linspace(-60, 340, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Start X Coordinate [cm]", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=9)
# plt.axvline(-40,color='k',linestyle='--',lw=3)
# plt.axvline(300,color='k',linestyle='--',lw=3)
plt.gca().axes.get_yaxis().set_ticks([])
plt.show()

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["_trk_start_z"].values,
    bins=np.linspace(500, 1050, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Start Z Coordinate [cm]", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=3)
# plt.axvline(697,color='k',linestyle='--',lw=3)
# plt.axvline(740,color='k',linestyle='--',lw=3)
plt.gca().axes.get_yaxis().set_ticks([])
plt.show()

In [ ]:
ydiff_all = df["_trk_end_y"].values - df["_trk_start_y"].values

fig = plt.figure(figsize=(6, 6))
plt.hist(
    ydiff_all,
    bins=np.linspace(-200, 200, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Y Length [cm]", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=1)
plt.yscale("log")
plt.gca().axes.get_yaxis().set_ticks([])
plt.show()

xdiff_all = np.abs(df["_trk_end_x"].values - df["_trk_start_x"].values)

fig = plt.figure(figsize=(6, 6))
plt.hist(
    xdiff_all,
    bins=np.linspace(0, 260, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("X Length [cm]", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=1)
# plt.yscale('log')
plt.gca().axes.get_yaxis().set_ticks([])
plt.show()

lendiff_all = df["SElen"].values / df["_trk_len"].values

fig = plt.figure(figsize=(6, 6))
plt.hist(
    lendiff_all,
    bins=np.linspace(0, 1.1, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("SE len / Trk len [cm]", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=2)
plt.yscale("log")
plt.gca().axes.get_yaxis().set_ticks([])
plt.show()

In [ ]:
BINSP = np.linspace(-1, 1, 100)

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["_px"].values,
    bins=BINSP,
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Px", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=1)
plt.gca().axes.get_yaxis().set_ticks([])
plt.show()

BINSY = np.linspace(-1, 0, 100)

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["_py"].values,
    bins=BINSY,
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Py", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=9)
plt.gca().axes.get_yaxis().set_ticks([])
plt.show()

fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["_pz"].values,
    bins=BINSP,
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Pz", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=3)
plt.gca().axes.get_yaxis().set_ticks([])
plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.hist(
    df["_nhit_endpoint"].values,
    bins=np.linspace(0, 100, 100),
    histtype="step",
    lw=2,
    color="b",
    normed=True,
    label="all $\mu$",
)
plt.xlabel("Points  [cm]", fontsize=20, fontweight="bold")
plt.grid()
plt.legend(loc=1)
# plt.axvline(697,color='k',linestyle='--',lw=3)
# plt.axvline(740,color='k',linestyle='--',lw=3)
plt.yscale("log")
# plt.gca().axes.get_yaxis().set_ticks([])
plt.show()

In [ ]:
print df.shape[0]
# print df['_nhit_endpoint']

# study impact of cuts
cut_v = ['BA_000_050 != -1 and BA_050_100 != -1',\
         '_trk_end_y > -98',\
         #'_trk_end_y > -80',\
         'dt > 15',\
         #'_trk_end_z < 700',\
         '_trk_end_z < 697 or _trk_end_z > 745',\
         '_trk_end_x > -40 and _trk_end_x < 300',\
         '_trk_end_x < 255 or _trk_end_x > 258',\
         '_trk_end_y < _trk_start_y',\
        'abs(_trk_end_x - _trk_start_x) < 150',\
         #'_trk_end_z < 700',\
         'BA_000_025 > 3 and BA_000_025 < 20',\
         #'BA_000_050 > 10 and BA_000_050 < 20',\
         #'mipfit > 1.5 and mipfit < 2.0',\
         'mipfit > 1.3 and  mipfit < 2.1',\
         ]

dfcuts = df

for cut in cut_v:
    dfcuts = dfcuts.query(cut)
    # print("cut: %s \t %d"%(cut, len(dfcuts)))
    print("cut: {:<40} => {:>10}".format(cut, dfcuts.shape[0]))    

In [ ]:
rr_cumulative_v   = []
dqdx_cumulative_v = []
dqdx_trunc_cumulative_v = []
pitch_cumulative_v = []
pz_cumulative_v = []
py_cumulative_v = []
px_cumulative_v = []

poly_0 = []
dmin_v = []

print dfcuts.shape

for idx, entry in dfcuts.iterrows():
    
    rr_v   = np.array(entry['_rr_y'])
    dqdx_v = np.array(entry['_dqdx_y'])
    dqdx_trunc_v = np.array(entry['_dqdx_tm_y'])
    pz = entry['_pz']
    py = entry['_py']
    px = entry['_px']
    pitch = entry['_pitch_y'] #0.3/np.abs(pz)
    
    
    for dqdx in dqdx_v:
        dqdx_cumulative_v.append(dqdx)
    for dqdx in dqdx_trunc_v:
        dqdx_trunc_cumulative_v.append(dqdx)
    for rr in rr_v:
        rr_cumulative_v.append(rr)
        pitch_cumulative_v.append(pitch)
        px_cumulative_v.append(px)
        py_cumulative_v.append(py)
        pz_cumulative_v.append(pz)

In [ ]:
fig = plt.figure(figsize=(12, 6))
BINS = (np.linspace(0, 200, 100), np.linspace(150, 500, 100))
plt.hist2d(rr_cumulative_v, dqdx_cumulative_v, bins=BINS)
plt.xlabel("Residual Range [cm]", fontsize=20, fontweight="bold")
plt.ylabel("dQ/dx [ADC / cm]", fontsize=20, fontweight="bold")
# plt.title('dQ/dx vs. RR for selected tracks [cosmic MC sample]',fontweight='bold',fontsize=18)
plt.title(
    "dQ/dx vs. RR for selected tracks [Run 4a Cosmic Data]",
    fontweight="bold",
    fontsize=18,
)
plt.grid()
plt.savefig("./figs/all_rr_v_dqdx.png", bbox_inches="tight")
plt.show()

In [ ]:
# save to picke file:
fout = open("stopmu_mcc9_data.csv", "wb")

for i, rr in enumerate(rr_cumulative_v):
    px = px_cumulative_v[i]
    py = py_cumulative_v[i]
    pz = pz_cumulative_v[i]
    fout.write(
        "%.02f \t %.02f \t %.02f \t %.02f \t %.02f \t %.02f \n"
        % (rr, dqdx_cumulative_v[i], pitch_cumulative_v[i], px, py, pz)
    )

fout.close()

In [ ]:
# VALIDATE RANGE vs. CALORIMETRY ENERGY

In [ ]:
# invert Recombination Modified Box Model to get dE/dx from dQ/dx

# argon density [g/cm^3]
rho = 1.396
# electric field [kV/cm]
efield = 0.273
# ionization energy [MeV/e-]
Wion = 23.6 * (10 ** (-6))

fModBoxA = 0.93
fModBoxB = 0.562


def ModBoxInverse(dqdx):
    dedx = (np.exp(fModBoxB * Wion * dqdx) - fModBoxA) / fModBoxB
    return dedx

In [ ]:
from scipy.interpolate import spline

pdg_rr_v = []
pdg_dedx_v = []
pdg_ke_v = []
pdg_rr_fit_v = []
pdg_dedx_fit_v = []
pdg_ke_fit_v = []

# fin = open('/home/david/Neutrinos/StopMuCalibration/mutable.txt')
fin = open('./data/mutable.txt')
rho = 1.396

for line in fin:
    words = line.split()
    pdg_rr_v.append(   float(words[-1])/rho )
    pdg_dedx_v.append( float(words[2]) *rho )
    pdg_ke_v.append(   float(words[0])      )


pdg_rr_fit_v   = np.linspace(1,1000,10000)
pdg_dedx_fit_v = spline(pdg_rr_v,pdg_dedx_v,pdg_rr_fit_v)
pdg_ke_fit_v = spline(pdg_rr_v,pdg_ke_v,pdg_rr_fit_v)


# fiven a RR value, get the energy
def ERange(RR):
    for i in xrange(len(pdg_rr_fit_v)):
        if (RR <= pdg_rr_fit_v[i]):
            return pdg_ke_fit_v[i]              
    return -1

print ERange(76.14)

fig = plt.figure(figsize=(6,6))
plt.plot(pdg_ke_v,pdg_rr_v,'ro')
plt.grid()
plt.plot(pdg_ke_fit_v,pdg_rr_fit_v,'b--')
plt.xlim([8,2000])
plt.ylim([0.9,1000])
plt.xlabel('KE [MeV]')
plt.ylabel('Range [cm]')
plt.xscale('log')
plt.yscale('log')
plt.savefig("./figs/pdg_ke_v_range.pdf", bbox_inches="tight")
plt.savefig("./figs/pdg_ke_v_range.png", bbox_inches="tight")
plt.show()

In [ ]:
def Pitch(x):
    return 0.3 / np.abs(x["_pz"])


dfcuts["pitch"] = dfcuts.apply(lambda x: Pitch(x), axis=1)

In [ ]:
def Energy(x, elecgain):
    energy = 0.0
    dqdx_v = x["_dqdx_y"]
    rr_v = x["_rr_y"]
    for i in xrange(1, len(dqdx_v)):
        dqdx = dqdx_v[i]
        drr = np.abs(rr_v[i - 1] - rr_v[i])
        dedx = ModBoxInverse(dqdx * elecgain)
        energy += drr * dedx
    energy += (np.abs(rr_v[0] - rr_v[1])) * ModBoxInverse(dqdx_v[0] * elecgain)
    return energy


# applied_gain = 238.0
applied_gain = 242.6

# dfcuts["energy"] = dfcuts.apply(lambda x: Energy(x, 238.0), axis=1)
# use later calculated elecgain value from Run4a data: 242.6
dfcuts["energy"] = dfcuts.apply(lambda x: Energy(x, applied_gain), axis=1)

In [ ]:
def EnergyfromRange(x):
    rrmax = np.max(x["_rr_y"])
    return ERange(rrmax)


dfcuts["erange"] = dfcuts.apply(lambda x: EnergyfromRange(x), axis=1)

In [ ]:
def gauss(x, mu, sigma, A):
    return A * (
        1.0
        / np.sqrt(np.pi * 2 * sigma * sigma)
        * np.exp(-((x - mu) ** 2) / (2 * sigma * sigma))
    )

In [ ]:
NSKIP = 15

In [ ]:
fig = plt.figure(figsize=(6, 6))
BINS = (np.linspace(200, 800, 50), np.linspace(200, 800, 50))
plt.hist2d(dfcuts["erange"].values, dfcuts["energy"].values, bins=BINS)
plt.grid()
plt.ylabel("Range Energy [MeV]", fontsize=20, fontweight="bold")
plt.xlabel("Calorimetry Energy [MeV]", fontsize=20, fontweight="bold")
plt.xlim([300, 800])
plt.ylim([300, 800])
plt.title(PLOTSTATUS, fontsize=14, loc="right")
plt.title("Gain = $%0.1f$ [e/ADC]" % applied_gain, fontsize=14, loc="left")
plt.savefig("./figs/range_v_calo.png", bbox_inches="tight")
plt.savefig("./figs/range_v_calo.pdf", bbox_inches="tight")
plt.show()

fig = plt.figure(figsize=(6, 6))
# dfratio = dfcuts.query('energy/_energy_mc > 0 and energy/_energy_mc < 1.5')
frac_v = dfcuts["energy"].values / dfcuts["erange"].values


vals, bine = np.histogram(frac_v, bins=np.linspace(0.5, 1.5, 50))
binc = 0.5 * (bine[1:] + bine[:-1])
popt, popv = curve_fit(
    gauss, binc[NSKIP:-NSKIP], vals[NSKIP:-NSKIP], p0=[1.0, 0.1, np.sum(vals)]
)
xvals = np.linspace(binc[NSKIP], binc[-NSKIP], 1000)

fig = plt.figure(figsize=(6, 6))
plt.plot(
    xvals,
    gauss(xvals, *popt),
    "r--",
    lw=3,
    # label="$\mu$ = %.03f $\sigma$ = %.03f" % (popt[0], popt[1]),
    label="$\mu$ = %.03f $\pm$ %.03f\n$\sigma$ = %.03f $\pm$ %.03f"
    % (popt[0], np.sqrt(popv[0][0]), popt[1], np.sqrt(popv[1][1])),
)
plt.errorbar(binc, vals, yerr=np.sqrt(vals), fmt="bo", lw=2, markersize=8)
# plt.hist(frac_v,bins=np.linspace(0.7,1.3,50),histtype='step',lw=2)
plt.xlim([0.5, 1.5])
plt.ylim([0, np.max(vals) * 1.3])
plt.grid()
plt.legend(fontsize=16)
plt.ylabel("Number of Tracks", fontsize=20, fontweight="bold")
plt.xlabel("Calorimetry Energy / Range Energy", fontsize=20, fontweight="bold")
plt.title(PLOTSTATUS, fontsize=14, loc="right")
plt.title("Gain = $%0.1f$ [e/ADC]" % applied_gain, fontsize=14, loc="left")
plt.savefig("./figs/range_calo_ratio.pdf", bbox_inches="tight")
plt.savefig("./figs/range_calo_ratio.png", bbox_inches="tight")
plt.show()

pull_v = (dfcuts["energy"].values - dfcuts["erange"].values) / dfcuts["erange"].values

vals, bine = np.histogram(pull_v, bins=np.linspace(-0.5, 0.5, 50))
binc = 0.5 * (bine[1:] + bine[:-1])
popt, popv = curve_fit(
    gauss, binc[NSKIP:-NSKIP], vals[NSKIP:-NSKIP], p0=[0.0, 0.1, np.sum(vals)]
)
xvals = np.linspace(binc[NSKIP], binc[-NSKIP], 1000)

fig = plt.figure(figsize=(6, 6))
plt.plot(
    xvals,
    gauss(xvals, *popt),
    "r--",
    lw=3,
    # label="$\mu$ = %.03f $\sigma$ = %.03f" % (popt[0], popt[1]),
    label="$\mu$ = %.03f $\pm$ %.03f\n$\sigma$ = %.03f $\pm$ %.03f"
    % (popt[0], np.sqrt(popv[0][0]), popt[1], np.sqrt(popv[1][1])),
)
plt.errorbar(binc, vals, yerr=np.sqrt(vals), fmt="bo", lw=2, markersize=8)
# plt.hist(frac_v,bins=np.linspace(0.7,1.3,50),histtype='step',lw=2)
plt.xlim([-0.5, 0.5])
plt.ylim([0, np.max(vals) * 1.3])
plt.grid()
plt.legend(fontsize=16)
plt.ylabel("Number of Tracks", fontsize=20, fontweight="bold")
plt.xlabel("(Calorimetry - Range) / Range Energy", fontsize=20, fontweight="bold")
plt.title(PLOTSTATUS, fontsize=14, loc="right")
plt.title("Gain = $%0.1f$ [e/ADC]" % applied_gain, fontsize=14, loc="left")
# plt.title("[Run 4a EXT Validation]", fontsize=14, loc="right")
plt.savefig("./figs/range_calo_pull.pdf", bbox_inches="tight")
plt.savefig("./figs/range_calo_pull.png", bbox_inches="tight")
plt.show()